In [1]:
# Third party imports
from IPython.display import display, HTML
# Local imports
from libcomcat.dataframes import (get_detail_data_frame, get_dyfi_data_frame,
                                  get_history_data_frame, get_magnitude_data_frame,
                                  get_pager_data_frame, get_phase_dataframe,
                                  get_summary_data_frame)
from libcomcat.search import search, get_event_by_id


In [2]:
import pandas as pd

(Returns NoneType if no pager product)

In [3]:
df = pd.read_csv('../DASHBOARD/CSV_ORIGINAL/raw_alert_usa.csv')

In [11]:
df["properties.type"].unique()

array(['earthquake'], dtype=object)

In [10]:
df.loc[df["properties.alert"] != 'green']

,type,id,properties.mag,properties.place,properties.time,properties.updated,properties.tz,properties.url,properties.detail,properties.felt,...,properties.rms,properties.gap,properties.magType,properties.type,properties.title,geometry.type,geometry.coordinates,Longitud,Latitud,Profundidad
1216,Feature,nc73827571,5.35,"15km SE of Rio Dell, CA",2023-01-01 18:35:04.510,2023-04-05 15:47:32.501,NaN,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,975.0,...,0.13,44.0,mw,earthquake,"M 5.4 - 15km SE of Rio Dell, CA",Point,"[-123.971, 40.409, 30.63]",-123.971000,40.409000,30.630
1217,Feature,nc73666231,6.20,"7km N of Petrolia, CA",2021-12-20 20:10:31.310,2022-12-30 01:11:35.632,NaN,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,4407.0,...,0.13,170.0,mw,earthquake,"M 6.2 - 7km N of Petrolia, CA",Point,"[-124.298, 40.3901667, 27.04]",-124.298000,40.390167,27.040
1218,Feature,se60324281,5.10,"4 km SE of Sparta, North Carolina",2020-08-09 12:07:37.650,2022-08-07 19:29:16.653,NaN,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,66710.0,...,0.38,50.0,mw,earthquake,"M 5.1 - 4 km SE of Sparta, North Carolina",Point,"[-81.0865, 36.4743333, 4.14]",-81.086500,36.474333,4.140
1219,Feature,uu60363602,5.70,"5 km NNE of Magna, Utah",2020-03-18 13:09:31.530,2023-04-13 01:37:46.041,NaN,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,26368.0,...,0.20,50.0,mw,earthquake,"M 5.7 - 5 km NNE of Magna, Utah",Point,"[-112.0783333, 40.751, 11.9]",-112.078333,40.751000,11.900
1220,Feature,ci38457511,7.10,"18km W of Searles Valley, CA",2019-07-06 03:19:53.040,2023-05-05 03:30:11.778,-480.0,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,24391.0,...,0.22,43.0,mw,earthquake,"M 7.1 - 18km W of Searles Valley, CA",Point,"[-117.5993333, 35.7695, 8]",-117.599333,35.769500,8.000
1221,Feature,ci38443183,6.40,Ridgecrest Earthquake Sequence,2019-07-04 17:33:49.000,2023-03-30 06:27:25.574,NaN,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,27543.0,...,0.16,47.0,mw,earthquake,M 6.4 - Ridgecrest Earthquake Sequence,Point,"[-117.5038333, 35.7053333, 10.5]",-117.503833,35.705333,10.500
1222,Feature,ci15481673,5.10,"2km NW of Brea, CA",2014-03-29 04:09:42.170,2022-11-05 22:39:58.701,NaN,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,6312.0,...,0.24,9.0,mw,earthquake,"M 5.1 - 2km NW of Brea, CA",Point,"[-117.9158333, 33.9325, 5.09]",-117.915833,33.932500,5.090
1223,Feature,ci15199681,5.32,"5km NNW of Brawley, CA",2012-08-26 19:31:23.040,2022-08-06 11:20:16.943,NaN,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,3050.0,...,0.21,73.0,mw,earthquake,"M 5.3 - 5km NNW of Brawley, CA",Point,"[-115.5536667, 33.0171667, 8.323]",-115.553667,33.017167,8.323
1224,Feature,se609212,5.80,"11 km SSW of Mineral, Virginia",2011-08-23 17:51:04.250,2023-03-25 22:31:42.847,NaN,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,131619.0,...,0.48,80.0,mw,earthquake,"M 5.8 - 11 km SSW of Mineral, Virginia",Point,"[-77.9363333, 37.9096667, 6]",-77.936333,37.909667,6.000
1225,Feature,ci14607924,5.38,"12km WNW of Alberto Oviedo Mota, B.C., MX",2010-04-04 23:25:07.190,2022-05-02 21:50:24.255,NaN,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,...,0.30,278.0,mw,earthquake,"M 5.4 - 12km WNW of Alberto Oviedo Mota, B.C., MX",Point,"[-115.2925, 32.2661667, 9.987]",-115.292500,32.266167,9.987


In [6]:
import json

In [12]:
df["properties.alert"].unique()

array(['green', 'yellow', 'orange', 'red'], dtype=object)

In [9]:
extra_data = []
for id in df['id']:
    pager_event = get_event_by_id(id)
    pager_country_df = get_pager_data_frame(pager_event, get_losses=True, get_country_exposures=False, get_all_versions=False)
    
    if pager_country_df is not None:
        try:
            pager_country_json = pager_country_df.to_json()
            pager_country_data = json.loads(pager_country_json)
            extra_data.append(pager_country_data)
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON for id {id}: {e}")
            continue

extra_data_df = pd.concat(extra_data)

df = pd.merge(df, extra_data_df, on='id', how='left')
df.head(1)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)